Linear Regression using pytorch builtins


In [2]:
import torch.nn as nn
import numpy as np

In [3]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

In [5]:
# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                   dtype='float32')



In [11]:
import torch
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

Dataset and DataLoader

In [7]:
##We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.
from torch.utils.data import TensorDataset

In [12]:
##dataset
train_ds=TensorDataset(inputs, targets)
train_ds[0:3]


(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [13]:
from torch.utils.data import DataLoader

In [14]:
batch_size=5
train_dl=DataLoader(train_ds, batch_size, shuffle=True)

In [15]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In each iteration, the data loader returns one batch of data with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

In [16]:
##xb=batch of inputs
##yb batch of iutputs or targets
for xb, yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[ 88., 134.,  59.],
        [ 74.,  66.,  43.],
        [101.,  44.,  37.],
        [ 91.,  88.,  64.],
        [ 87., 135.,  57.]])
tensor([[118., 132.],
        [ 57.,  69.],
        [ 21.,  38.],
        [ 81., 101.],
        [118., 134.]])


Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [17]:
## 3 is number of inputs and 2 is number of ouutputs
model=nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.0020, -0.1028,  0.2266],
        [-0.1172, -0.5666, -0.0608]], requires_grad=True)
Parameter containing:
tensor([-0.0239,  0.4980], requires_grad=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix

In [19]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0020, -0.1028,  0.2266],
         [-0.1172, -0.5666, -0.0608]], requires_grad=True),
 Parameter containing:
 tensor([-0.0239,  0.4980], requires_grad=True)]

In [20]:
preds=model(inputs)

In [21]:
preds

tensor([[  2.9784, -48.6326],
        [  5.6144, -63.9170],
        [ -0.4823, -89.1469],
        [  4.1440, -38.0682],
        [  6.1077, -66.2363],
        [  3.0832, -48.1832],
        [  5.9438, -63.4112],
        [ -0.2537, -89.3249],
        [  4.0392, -38.5176],
        [  6.3323, -66.1799],
        [  3.3078, -48.1268],
        [  5.7192, -63.4676],
        [ -0.8118, -89.6527],
        [  3.9194, -38.1246],
        [  6.0029, -66.6857]], grad_fn=<AddmmBackward0>)

In [22]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [23]:
import torch.nn.functional as F

In [24]:
loss_fn=F.mse_loss

In [25]:
loss=loss_fn(model(inputs), targets)
print(loss)

tensor(16283.3789, grad_fn=<MseLossBackward0>)


In [27]:
 opt=torch.optim.SGD(model.parameters(), lr=1e-5)
 #stochastisc gradient descent as data is selected in random batch

In [29]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
  for epoch in range(num_epochs):
    #train with batches
    for xb, yb in train_dl:
      #generate predictions
      pred=model(xb)
      #calculate loss
      loss=loss_fn(pred, yb)
      # compute gradients
      loss.backward()
      #update parameters using gradients
      opt.step()
      #repeate gradient to zero
      opt.zero_grad()
    if(epoch+1)%10==0:
      print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [30]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 291.9304
Epoch [20/100], Loss: 292.7029
Epoch [30/100], Loss: 199.9381
Epoch [40/100], Loss: 184.2863
Epoch [50/100], Loss: 99.1861
Epoch [60/100], Loss: 71.5500
Epoch [70/100], Loss: 78.5750
Epoch [80/100], Loss: 27.9972
Epoch [90/100], Loss: 44.2457
Epoch [100/100], Loss: 16.8910


In [31]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.2482,  71.9949],
        [ 82.1374,  99.4412],
        [116.5926, 133.0208],
        [ 27.7298,  46.1063],
        [ 97.9526, 111.7954],
        [ 57.1761,  71.0886],
        [ 81.9087,  99.2424],
        [116.8914, 133.5135],
        [ 28.8019,  47.0126],
        [ 98.7959, 112.5029],
        [ 58.0195,  71.7961],
        [ 81.0653,  98.5349],
        [116.8213, 133.2197],
        [ 26.8864,  45.3988],
        [ 99.0247, 112.7017]], grad_fn=<AddmmBackward0>)

In [32]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [33]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[55.0754, 69.1845]], grad_fn=<AddmmBackward0>)